In [2]:
import pandas as pd
import numpy as np
import pyreadstat
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('bankruptcy_Train.csv')

Data fields <br>
attr1 - net profit / total assets <br>
attr2 - total liabilities / total assets<br>
attr3 - working capital / total assets<br>
attr4 - current assets / short-term liabilities<br>
attr5 - [(cash + short-term securities + receivables - short-term liabilities) / (operating expenses - depreciation)] * 365<br>
attr6 - retained earnings / total assets<br>
attr7 - EBIT / total assets<br>
attr8 - book value of equity / total liabilities<br>
attr9 - sales / total assets<br>
attr10 - equity / total assets<br>
attr11 - (gross profit + extraordinary items + financial expenses) / total assets<br>
attr12 - gross profit / short-term liabilities<br>
attr13 - (gross profit + depreciation) / sales<br>
attr14 - (gross profit + interest) / total assets<br>
attr15 - (total liabilities * 365) / (gross profit + depreciation)<br>
attr16 - (gross profit + depreciation) / total liabilities<br>
attr17 - total assets / total liabilities<br>
attr18 - gross profit / total assets<br>
attr19 - gross profit / sales<br>
attr20 - (inventory * 365) / sales<br>
attr21 - sales (n) / sales (n-1)<br>
attr22 - profit on operating activities / total assets<br>
attr23 - net profit / sales<br>
attr24 - gross profit (in 3 years) / total assets<br>
attr25 - (equity - share capital) / total assets<br>
attr26 - (net profit + depreciation) / total liabilities<br>
attr27 - profit on operating activities / financial expenses<br>
attr28 - working capital / fixed assets<br>
attr29 - logarithm of total assets<br>
attr30 - (total liabilities - cash) / sales<br>
attr31 - (gross profit + interest) / sales<br>
attr32 - (current liabilities * 365) / cost of products sold<br>
attr33 - operating expenses / short-term liabilities<br>
attr34 - operating expenses / total liabilities<br>
attr35 - profit on sales / total assets<br>
attr36 - total sales / total assets<br>
attr37 - (current assets - inventories) / long-term liabilities<br>
attr38 - constant capital / total assets<br>
attr39 - profit on sales / sales<br>
attr40 - (current assets - inventory - receivables) / short-term liabilities<br>
attr41 - total liabilities / ((profit on operating activities + depreciation) * (12/365))<br>
attr42 - profit on operating activities / sales<br>
attr43 - rotation receivables + inventory turnover in days<br>
attr44 - (receivables * 365) / sales<br>
attr45 - net profit / inventory<br>
attr46 - (current assets - inventory) / short-term liabilities<br>
attr47 - (inventory * 365) / cost of products sold<br>
attr48 - EBITDA (profit on operating activities - depreciation) / total assets<br>
attr49 - EBITDA (profit on operating activities - depreciation) / sales<br>
attr50 - current assets / total liabilities<br>
attr51 - short-term liabilities / total assets<br>
attr52 - (short-term liabilities * 365) / cost of products sold)<br>
attr53 - equity / fixed assets<br>
attr54 - constant capital / fixed assets<br>
attr55 - working capital<br>
attr56 - (sales - cost of products sold) / sales<br>
attr57 - (current assets - inventory - short-term liabilities) / (sales - gross profit - depreciation)<br>
attr58 - total costs /total sales<br>
attr59 - long-term liabilities / equity<br>
attr60 - sales / inventory<br>
attr61 - sales / receivables<br>
attr62 - (short-term liabilities *365) / sales<br>
attr63 - sales / short-term liabilities <br>
attr64 - sales / fixed assets<br>
class - the response variable Y: 0 = did not bankrupt; 1 = bankrupt<br>

In [11]:
df['class'].mean() #сколько банкротилось из 10к

0.0203

In [13]:
df.describe()

,Attr1,Attr2,Attr3,Attr4,Attr5,Attr6,Attr7,Attr8,Attr9,Attr10,...,Attr56,Attr57,Attr58,Attr59,Attr60,Attr61,Attr62,Attr63,Attr64,class
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.007954,-0.007140,-0.003544,0.005005,-0.004687,0.001455,-0.006963,0.009053,0.006763,0.007456,...,0.009804,0.005208,-0.009751,-0.002655,0.004548,0.014331,-0.006864,0.020750,-0.003984,0.020300
std,1.396405,1.409440,1.015494,0.908020,1.394750,1.286713,1.412509,1.064426,1.171199,1.407349,...,0.017370,0.718682,0.026124,0.467139,1.252571,1.171160,0.277288,1.115182,0.930875,0.141032
min,-9.474787,-140.604555,-25.597146,-0.381641,-138.720013,-26.249562,-141.176615,-0.966015,-1.294340,-2.528495,...,-1.657288,-42.381245,-0.037484,-3.218490,-0.048598,-0.412525,-0.022204,-0.743297,-0.050969,0.000000
25%,-0.079776,-0.055604,-0.516971,-0.192306,0.006798,-0.016047,-0.008018,-0.445667,-0.388184,-0.068584,...,0.009315,0.003199,-0.010766,-0.022740,-0.043418,-0.221237,-0.016847,-0.371384,-0.041603,0.000000
50%,-0.019204,0.000246,0.003186,-0.117972,0.007302,-0.016047,0.004280,-0.260150,-0.307842,0.000253,...,0.009838,0.010013,-0.009924,-0.020438,-0.039928,-0.139653,-0.013821,-0.180311,-0.034319,0.000000
75%,0.081218,0.056957,0.585722,0.022705,0.007753,0.016026,0.024222,0.098874,0.156638,0.070036,...,0.010693,0.019900,-0.009416,-0.014401,-0.031820,-0.002210,-0.009476,0.133244,-0.020507,0.000000
max,137.557872,2.004925,3.324327,60.825460,11.556238,121.354736,0.623095,47.504463,62.936396,140.123299,...,0.036727,37.405312,2.551674,27.969785,117.341069,59.139158,23.970250,62.527240,90.774695,1.000000
